In [18]:
# data processing
import numpy as np
import pandas as pd 

# machine learning
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# utils
import time
from datetime import timedelta

# some configuratin flags and variables
verbose=0 # Use in classifier

# Input files
file_train='C:\\Studenci\\Biostatystyka I mgr\\AJ\\randomForrest_3\\train.csv'
file_test='C:\\Studenci\\Biostatystyka I mgr\\AJ\\randomForrest_3\\test.csv'

# defeine random seed for reproducibility
seed = 69
np.random.seed(seed)

# read training data
train_df = pd.read_csv(file_train,index_col='PassengerId')

In [3]:
def prep_data(df):
    # Drop unwanted features
    df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
    
    # Fill missing data: Age and Fare with the mean, Embarked with most frequent value
    df[['Age']] = df[['Age']].fillna(value=df[['Age']].mean())
    df[['Fare']] = df[['Fare']].fillna(value=df[['Fare']].mean())
    df[['Embarked']] = df[['Embarked']].fillna(value=df['Embarked'].value_counts().idxmax())
    
    # Convert categorical  features into numeric
    df['Sex'] = df['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
      
    # Convert Embarked to one-hot
    enbarked_one_hot = pd.get_dummies(df['Embarked'], prefix='Embarked')
    df = df.drop('Embarked', axis=1)
    df = df.join(enbarked_one_hot)

    return df

In [4]:
train_df = prep_data(train_df)
train_df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [5]:
# X contains all columns except 'Survived'  
X = train_df.drop(['Survived'], axis=1).values.astype(float)

# It is almost always a good idea to perform some scaling of input values when using neural network models (jb).

scale = StandardScaler()
X = scale.fit_transform(X)

# Y is just the 'Survived' column
Y = train_df['Survived'].values

In [22]:
    def create_model(optimizer='adam', init='uniform'):
        # create model
        if verbose: print("**Create model with optimizer: %s; init: %s" % (optimizer, init) )
        model = Sequential()
        model.add(Dense(120, input_dim=X.shape[1], kernel_initializer=init, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(60, kernel_initializer=init, activation='selu'))
        model.add(Dropout(0.2))
        model.add(Dense(60, kernel_initializer=init, activation='selu'))
        model.add(Dropout(0.2))
        model.add(Dense(30, kernel_initializer=init, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return model

In [7]:
run_gridsearch = False

if run_gridsearch:
    
    start_time = time.time()
    if verbose: print (time.strftime( "%H:%M:%S " + "GridSearch started ... " ) )
    optimizers = ['rmsprop', 'adam']
    inits = ['glorot_uniform', 'normal', 'uniform']
    epochs = [50, 100, 200, 400]
    batches = [5, 10, 20]
    
    model = KerasClassifier(build_fn=create_model, verbose=verbose)
    
    param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=inits)
    grid = GridSearchCV(estimator=model, param_grid=param_grid)
    grid_result = grid.fit(X, Y)
    
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    if verbose: 
        for mean, stdev, param in zip(means, stds, params):
            print("%f (%f) with: %r" % (mean, stdev, param))
        elapsed_time = time.time() - start_time  
        print ("Time elapsed: ",timedelta(seconds=elapsed_time))
        
    best_epochs = grid_result.best_params_['epochs']
    best_batch_size = grid_result.best_params_['batch_size']
    best_init = grid_result.best_params_['init']
    best_optimizer = grid_result.best_params_['optimizer']
    
else:
    # pre-selected paramters
    best_epochs = 200
    best_batch_size = 5
    best_init = 'glorot_uniform'
    best_optimizer = 'rmsprop'

In [ ]:
# Read test data
test_df = pd.read_csv(file_test,index_col='PassengerId')
# Prep and clean data
test_df = prep_data(test_df)
# Create X_test
X_test = test_df.values.astype(float)
# Scaling
X_test = scale.transform(X_test)

In [ ]:
model = create_model()
model.fit(X, Y, batch_size=25, epochs=2000)

Epoch 1/2000
891/891 [==============================] - 0s 388us/step - loss: 0.6599 - accuracy: 0.6577
Epoch 2/2000
891/891 [==============================] - 0s 71us/step - loss: 0.4768 - accuracy: 0.8036
Epoch 3/2000
891/891 [==============================] - 0s 68us/step - loss: 0.4459 - accuracy: 0.8126
Epoch 4/2000
891/891 [==============================] - 0s 67us/step - loss: 0.4310 - accuracy: 0.8126
Epoch 5/2000
891/891 [==============================] - 0s 67us/step - loss: 0.4199 - accuracy: 0.8238
Epoch 6/2000
891/891 [==============================] - 0s 59us/step - loss: 0.4164 - accuracy: 0.8272
Epoch 7/2000
891/891 [==============================] - 0s 58us/step - loss: 0.4129 - accuracy: 0.8339
Epoch 8/2000
891/891 [==============================] - 0s 67us/step - loss: 0.4155 - accuracy: 0.8227
Epoch 9/2000
891/891 [==============================] - 0s 57us/step - loss: 0.4147 - accuracy: 0.8283
Epoch 10/2000
891/891 [==============================] - 0s 66us/step - 

891/891 [==============================] - 0s 53us/step - loss: 0.3587 - accuracy: 0.8530
Epoch 80/2000
891/891 [==============================] - 0s 51us/step - loss: 0.3616 - accuracy: 0.8519
Epoch 81/2000
891/891 [==============================] - 0s 50us/step - loss: 0.3631 - accuracy: 0.8530
Epoch 82/2000
891/891 [==============================] - 0s 51us/step - loss: 0.3653 - accuracy: 0.8429
Epoch 83/2000
891/891 [==============================] - 0s 53us/step - loss: 0.3792 - accuracy: 0.8474
Epoch 84/2000
891/891 [==============================] - 0s 51us/step - loss: 0.3662 - accuracy: 0.8451
Epoch 85/2000
891/891 [==============================] - 0s 51us/step - loss: 0.3598 - accuracy: 0.8440
Epoch 86/2000
891/891 [==============================] - 0s 51us/step - loss: 0.3661 - accuracy: 0.8507
Epoch 87/2000
891/891 [==============================] - 0s 51us/step - loss: 0.3637 - accuracy: 0.8519
Epoch 88/2000
891/891 [==============================] - 0s 49us/step - loss: 

891/891 [==============================] - 0s 62us/step - loss: 0.3453 - accuracy: 0.8563
Epoch 158/2000
891/891 [==============================] - 0s 66us/step - loss: 0.3391 - accuracy: 0.8563
Epoch 159/2000
891/891 [==============================] - 0s 51us/step - loss: 0.3441 - accuracy: 0.8519
Epoch 160/2000
891/891 [==============================] - 0s 56us/step - loss: 0.3397 - accuracy: 0.8575
Epoch 161/2000
891/891 [==============================] - 0s 60us/step - loss: 0.3372 - accuracy: 0.8519
Epoch 162/2000
891/891 [==============================] - 0s 55us/step - loss: 0.3467 - accuracy: 0.8485
Epoch 163/2000
891/891 [==============================] - 0s 58us/step - loss: 0.3428 - accuracy: 0.8575
Epoch 164/2000
891/891 [==============================] - 0s 54us/step - loss: 0.3519 - accuracy: 0.8519
Epoch 165/2000
891/891 [==============================] - 0s 56us/step - loss: 0.3359 - accuracy: 0.8563
Epoch 166/2000
891/891 [==============================] - 0s 53us/step

891/891 [==============================] - 0s 73us/step - loss: 0.3303 - accuracy: 0.8575
Epoch 236/2000
891/891 [==============================] - 0s 74us/step - loss: 0.3347 - accuracy: 0.8608
Epoch 237/2000
891/891 [==============================] - 0s 73us/step - loss: 0.3267 - accuracy: 0.8575
Epoch 238/2000
891/891 [==============================] - 0s 75us/step - loss: 0.3254 - accuracy: 0.8563
Epoch 239/2000
891/891 [==============================] - 0s 75us/step - loss: 0.3322 - accuracy: 0.8496
Epoch 240/2000
891/891 [==============================] - 0s 72us/step - loss: 0.3486 - accuracy: 0.8541
Epoch 241/2000
891/891 [==============================] - 0s 73us/step - loss: 0.3250 - accuracy: 0.8597
Epoch 242/2000
891/891 [==============================] - 0s 72us/step - loss: 0.3255 - accuracy: 0.8563
Epoch 243/2000
891/891 [==============================] - 0s 69us/step - loss: 0.3465 - accuracy: 0.8552
Epoch 244/2000
891/891 [==============================] - 0s 68us/step

891/891 [==============================] - 0s 57us/step - loss: 0.3177 - accuracy: 0.8608
Epoch 314/2000
891/891 [==============================] - 0s 59us/step - loss: 0.3270 - accuracy: 0.8608
Epoch 315/2000
891/891 [==============================] - 0s 50us/step - loss: 0.3257 - accuracy: 0.8575
Epoch 316/2000
891/891 [==============================] - 0s 49us/step - loss: 0.3225 - accuracy: 0.8541
Epoch 317/2000
891/891 [==============================] - 0s 53us/step - loss: 0.3281 - accuracy: 0.8530
Epoch 318/2000
891/891 [==============================] - 0s 51us/step - loss: 0.3262 - accuracy: 0.8586
Epoch 319/2000
891/891 [==============================] - 0s 50us/step - loss: 0.3130 - accuracy: 0.8552
Epoch 320/2000
891/891 [==============================] - 0s 49us/step - loss: 0.3216 - accuracy: 0.8586
Epoch 321/2000
891/891 [==============================] - 0s 49us/step - loss: 0.3268 - accuracy: 0.8519
Epoch 322/2000
891/891 [==============================] - 0s 51us/step

...

891/891 [==============================] - 0s 51us/step - loss: 0.2550 - accuracy: 0.8934
Epoch 1093/2000
891/891 [==============================] - 0s 50us/step - loss: 0.2838 - accuracy: 0.8822
Epoch 1094/2000
891/891 [==============================] - 0s 50us/step - loss: 0.2606 - accuracy: 0.8934
Epoch 1095/2000
891/891 [==============================] - 0s 49us/step - loss: 0.2622 - accuracy: 0.8911
Epoch 1096/2000
891/891 [==============================] - 0s 53us/step - loss: 0.2670 - accuracy: 0.8956
Epoch 1097/2000
891/891 [==============================] - 0s 51us/step - loss: 0.2617 - accuracy: 0.8956
Epoch 1098/2000
891/891 [==============================] - 0s 55us/step - loss: 0.2699 - accuracy: 0.8855
Epoch 1099/2000
891/891 [==============================] - 0s 49us/step - loss: 0.2781 - accuracy: 0.8822
Epoch 1100/2000
891/891 [==============================] - 0s 50us/step - loss: 0.2876 - accuracy: 0.8900
Epoch 1101/2000
891/891 [==============================] - 0s 

891/891 [==============================] - 0s 54us/step - loss: 0.2578 - accuracy: 0.8956
Epoch 1170/2000
891/891 [==============================] - 0s 54us/step - loss: 0.2572 - accuracy: 0.8934
Epoch 1171/2000
891/891 [==============================] - 0s 53us/step - loss: 0.2662 - accuracy: 0.8900
Epoch 1172/2000
891/891 [==============================] - 0s 50us/step - loss: 0.2743 - accuracy: 0.8878
Epoch 1173/2000
891/891 [==============================] - 0s 50us/step - loss: 0.2520 - accuracy: 0.8945
Epoch 1174/2000
891/891 [==============================] - 0s 49us/step - loss: 0.2693 - accuracy: 0.8900
Epoch 1175/2000
891/891 [==============================] - 0s 54us/step - loss: 0.2613 - accuracy: 0.8967
Epoch 1176/2000
891/891 [==============================] - 0s 51us/step - loss: 0.2659 - accuracy: 0.8900
Epoch 1177/2000
891/891 [==============================] - 0s 51us/step - loss: 0.2637 - accuracy: 0.8934
Epoch 1178/2000
891/891 [==============================] - 0s 

891/891 [==============================] - 0s 50us/step - loss: 0.2771 - accuracy: 0.8844
Epoch 1247/2000
891/891 [==============================] - 0s 54us/step - loss: 0.2675 - accuracy: 0.8923
Epoch 1248/2000
891/891 [==============================] - 0s 51us/step - loss: 0.2490 - accuracy: 0.9001
Epoch 1249/2000
891/891 [==============================] - 0s 51us/step - loss: 0.2529 - accuracy: 0.8945
Epoch 1250/2000
891/891 [==============================] - 0s 55us/step - loss: 0.2749 - accuracy: 0.8956
Epoch 1251/2000
891/891 [==============================] - 0s 51us/step - loss: 0.2624 - accuracy: 0.8911
Epoch 1252/2000
891/891 [==============================] - 0s 50us/step - loss: 0.2566 - accuracy: 0.8945
Epoch 1253/2000
891/891 [==============================] - 0s 50us/step - loss: 0.2486 - accuracy: 0.9001
Epoch 1254/2000
891/891 [==============================] - 0s 54us/step - loss: 0.2609 - accuracy: 0.8911
Epoch 1255/2000
891/891 [==============================] - 0s 

891/891 [==============================] - 0s 54us/step - loss: 0.2520 - accuracy: 0.9001
Epoch 1324/2000
891/891 [==============================] - 0s 56us/step - loss: 0.2622 - accuracy: 0.8934
Epoch 1325/2000
891/891 [==============================] - 0s 51us/step - loss: 0.2575 - accuracy: 0.8911
Epoch 1326/2000
891/891 [==============================] - 0s 54us/step - loss: 0.2607 - accuracy: 0.8979
Epoch 1327/2000
891/891 [==============================] - 0s 51us/step - loss: 0.2630 - accuracy: 0.9080
Epoch 1328/2000
891/891 [==============================] - 0s 50us/step - loss: 0.2591 - accuracy: 0.8878
Epoch 1329/2000
891/891 [==============================] - 0s 50us/step - loss: 0.2645 - accuracy: 0.8900
Epoch 1330/2000
891/891 [==============================] - 0s 49us/step - loss: 0.2594 - accuracy: 0.9046
Epoch 1331/2000
891/891 [==============================] - 0s 49us/step - loss: 0.2529 - accuracy: 0.8956
Epoch 1332/2000
891/891 [==============================] - 0s 

891/891 [==============================] - 0s 50us/step - loss: 0.2629 - accuracy: 0.8945
Epoch 1401/2000
891/891 [==============================] - 0s 50us/step - loss: 0.2668 - accuracy: 0.8990
Epoch 1402/2000
891/891 [==============================] - 0s 49us/step - loss: 0.2436 - accuracy: 0.9024
Epoch 1403/2000
891/891 [==============================] - 0s 53us/step - loss: 0.2619 - accuracy: 0.8833
Epoch 1404/2000
891/891 [==============================] - 0s 54us/step - loss: 0.2458 - accuracy: 0.8923
Epoch 1405/2000
891/891 [==============================] - 0s 50us/step - loss: 0.2527 - accuracy: 0.8945
Epoch 1406/2000
891/891 [==============================] - 0s 49us/step - loss: 0.2466 - accuracy: 0.9035
Epoch 1407/2000
891/891 [==============================] - 0s 51us/step - loss: 0.2488 - accuracy: 0.9024
Epoch 1408/2000
891/891 [==============================] - 0s 53us/step - loss: 0.2399 - accuracy: 0.9046
Epoch 1409/2000
891/891 [==============================] - 0s 

In [12]:
# Create a classifier with best parameters
model_pred = KerasClassifier(build_fn=create_model, optimizer=best_optimizer, init=best_init, epochs=best_epochs, batch_size=best_batch_size, verbose=verbose)
model_pred.fit(X, Y)

# Read test data
test_df = pd.read_csv(file_test,index_col='PassengerId')
# Prep and clean data
test_df = prep_data(test_df)
# Create X_test
X_test = test_df.values.astype(float)
# Scaling
X_test = scale.transform(X_test)

# Predict 'Survived'
prediction = model_pred.predict(X_test)
print(prediction)

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
